In [1]:
import os
import json
import gdown
import pandas as pd
import numpy as np
### Plot
import matplotlib.pyplot as plt
import seaborn as sns
### HTML
from bs4 import BeautifulSoup
import re
### Warnings
import warnings
### Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud


/var/folders/y9/5bvycrgx33315dhzcfzrvf9w0000gn/T/ipykernel_49847/3365911365.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
##tmdb dataset with movies before and after 1995
tmdb_df = pd.read_csv('TMDB_movie_dataset_v11.csv') #tmdb dataset we will be working with
tmdb_df.describe()


,id,vote_average,vote_count,revenue,runtime,budget,popularity
count,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06
mean,6.625906e+05,2.101624e+00,2.139664e+01,7.421149e+05,5.059751e+01,2.924065e+05,1.349021e+00
std,3.549163e+05,3.110503e+00,3.391106e+02,1.803934e+07,6.229370e+01,5.134357e+06,8.071102e+00
min,2.000000e+00,0.000000e+00,0.000000e+00,-1.200000e+01,-2.800000e+01,0.000000e+00,0.000000e+00
25%,3.705740e+05,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,6.000000e-01
50%,6.643460e+05,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01,0.000000e+00,6.000000e-01
75%,9.721775e+05,5.000000e+00,1.000000e+00,0.000000e+00,9.000000e+01,0.000000e+00,9.140000e-01
max,1.254286e+06,1.000000e+01,3.449500e+04,3.000000e+09,1.440000e+04,9.000000e+08,2.994357e+03


In [4]:
tmdb_df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili"
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,en,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin"
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish"
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian"


In [5]:
len(tmdb_df.columns)

23

In [6]:
tmdb_df['vote_count'].quantile(0.9)

6.0

In [7]:
# in order to create a robust key "title_year to join with the movielens dataset"
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'], errors='coerce')
tmdb_df['year'] = tmdb_df['release_date'].dt.year.astype('Int64').astype(str).replace('nan', '')
tmdb_df['title_year'] = tmdb_df.apply(lambda x: f"{x['title']} ({x['year']})" if x['year'] else x['title'], axis=1)
tmdb_df.head()


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,year,title_year
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili",2010,Inception (2010)
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,2014,Interstellar (2014)
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin",2008,The Dark Knight (2008)
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish",2009,Avatar (2009)
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian",2012,The Avengers (2012)


In [9]:
df_ratings = pd.read_csv('ratings.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp']) # ^same
df_ratings.head(),df_ratings.shape

FileNotFoundError: [Errno 2] No such file or directory: 'ratings.csv'

In [ ]:
df_users = pd.read_csv(file_path + 'users.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['UserID', 'Gender', 'Age', 'Occupation','ZipCode']) # ^same
df_users.head(), df_users.shape

In [ ]:
df_movies = pd.read_csv(file_path + 'movies.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['MovieID', 'Title', 'Genre']) # ^same
df_movies.head()

In [ ]:
df_ml_combined = pd.merge(pd.merge(df_ratings, df_movies, on='MovieID'), df_users, on='UserID') #this is the merged version of the MovieLens Dataset
df_ml_combined.head(), df_ml_combined.shape

In [ ]:
# fuzzy wuzzy
#now to finally merge the datasets
merged_df = pd.merge(df_ml_combined, tmdb_df, left_on='Title', right_on='title_year', how='left') #main dataset
merged_df.head()
#use Genres from movielens

In [ ]:
merged_df.shape

In [ ]:
print(merged_df.columns)
# Counting how many NA values are in the 'UserID' column
user_id_na_count = merged_df['UserID'].isna().sum()

print("Number of NA values in the UserID column:", user_id_na_count)
# Assuming 'df' is your DataFrame
important_columns = ['Title', 'Genre', 'overview', 'tagline', 'original_language', 'release_date', 'runtime', 'vote_average', 'vote_count', 'production_companies', 'production_countries', 'spoken_languages', 'year']

# Check for any NA values in these columns
na_columns = merged_df[important_columns].isna().any()

# Print columns with NA values
print("Columns with NA values:", na_columns[na_columns].index.tolist())
na_columns.shape


# Start of work

In [ ]:
## start of work
# only movies that eexist in both datasets
pre_2000 = merged_df.dropna()
pre_2000.columns


In [ ]:
pre_2000.shape

In [ ]:
#mostly english --> not significant
pre_2000['original_language'].value_counts()

In [ ]:
#Mostly usa and uk--> can drop
pre_2000['production_countries'].value_counts()

In [ ]:
pre_2000['production_companies'].value_counts()

In [ ]:
#preprocess
# dropped age, occupation, zipcode from movielens
to_keep = ['UserID', 'Rating', 'Gender',
       'id', 'title', 'vote_average',
       'vote_count', 'revenue', 'runtime', 'adult','budget',
       'overview', 'popularity', 'tagline',
       'genres', 'production_companies',
       'spoken_languages', 'year']

pre_2000_analysis = pre_2000[to_keep].rename(columns={'UserID': 'userId',
                                                      'Rating': 'rating',
                                                      'Gender':'gender'})

# analysis
# about 3.5x more M than F reviews
# multiplier to F ratings



gender_count = pre_2000_analysis['gender'].value_counts()
gender_count.plot(kind='bar')
# -1 or 3.55zzxcad
pre_2000_analysis['gender'] = pre_2000_analysis['gender'].apply(lambda x: -1 if x == 'M' else  gender_count['M']/gender_count['F'])

# pre_2000_analysis['adult'] = pre_2000_analysis['adult'].apply(lambda x: 1 if x else -1)

# gender rating ->  more than 0 --> more prefered by F and vice versa
pre_2000_analysis['gender_rating'] = pre_2000_analysis['gender'] * pre_2000_analysis['rating']


In [ ]:
#aggregate
import seaborn as sns
# each row is 1 movie
agg_analysis = pre_2000_analysis.groupby('id').agg({k: 'mean' if v in ('int64', 'float64') else 'first'
                          for k,v in pre_2000_analysis.dtypes[1:].items()})
agg_analysis.head()

In [ ]:
#correlation matrix for numrical cols
to_drop = ['rating', 'gender', 'id']
agg_analysis = agg_analysis.drop(columns = to_drop)

numerical_columns = ['vote_average', 'vote_count', 'revenue', 'runtime', 'budget',
                     'popularity', 'gender_rating']
num_analysis = agg_analysis[numerical_columns]
num_corr = num_analysis.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(num_corr, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
agg_analysis.shape

not much correlation between gender rating and other columns  
  
except popularity --> slight inverse rs?

In [ ]:
## applying the pref gender column to tmdb dataset
## first must clean tmdb
# 1 million rows of movies
print(f'{tmdb_df.shape[0]} rows of movies')
print(f"{tmdb_df['vote_count'].quantile(0.9)} votes in the 90th percentile")
# threshold --> how many votes a movie must have to be included in our rec sys
# can be changed
threshold = 100
to_keep = ['id', 'title', 'vote_average',
       'vote_count', 'revenue', 'runtime', 'adult','budget',
       'overview', 'popularity', 'tagline',
       'genres', 'production_companies',
       'spoken_languages', 'year']
tmdb_above_100 = tmdb_df[tmdb_df['vote_count'] >= 100]
tmdb_below_100 = tmdb_df[tmdb_df['vote_count'] < 100]
tmdb_above_100 = tmdb_above_100[to_keep]
tmdb_above_100.shape

In [ ]:
#word soup + tfidf
# remove commas from genres
agg_analysis['genres'] =agg_analysis['genres'].apply(lambda x: x.replace(',',''))

word_cols = ['overview', 'tagline',
       'genres', 'production_companies',
       'spoken_languages']
# TODO: shuffle word soup
agg_analysis['word_soup'] = agg_analysis[word_cols].sum(axis = 1)


tmdb_above_100['genres'] =tmdb_above_100['genres'].apply(lambda x: str(x).replace(',',''))
tmdb_above_100[word_cols] = tmdb_above_100[word_cols].astype(str)
# can do more text preprocessing on the word soup --> remove all punctuations,
# lemmatize, stemming
tmdb_above_100['word_soup'] = tmdb_above_100[word_cols].sum(axis = 1)
tmdb_above_100['word_soup'].head()
# remove word columns since we are done with it
tmdb_above_100 = tmdb_above_100.drop(word_cols, axis = 1)
# edit adult column
agg_analysis['word_soup'].head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# can choose another way to encode the word soup e.g. BERT
tfidf = TfidfVectorizer(stop_words= 'english', max_features=100000,dtype = np.float32)
# tmdb_above_100 has 180000 rows --> bigger vocab
# agg_analysis (movielens and tmdb combined) has 216 rows --> smaller vocab
tfidf.fit(tmdb_above_100['word_soup'])
soup_tfidf = tfidf.transform(agg_analysis['word_soup'])

# print(soup_tfidf.shape,type(soup_tfidf))
# data for our classifier
_soup_tfidf = soup_tfidf.astype('float32').toarray()
# 1 for F, 0 for M
# label for our classifier
gender_pref = agg_analysis['gender_rating'].apply(lambda x: 1 if x > 0 else 0)




In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(soup_tfidf, gender_pref, test_size=0.2, random_state=42)

In [ ]:
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
# trying different models
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(X_train, y_train)

svc_clf = SVC()
svc_clf.fit(X_train, y_train)

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)

nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

# Make predictions on the testing set
svc_pred = svc_clf.predict(X_test)
lr_pred = lr_clf.predict(X_test)
nb_pred = nb_clf.predict(X_test)
xgb_pred = xgb_clf.predict(X_test)

svc_accuracy = accuracy_score(y_test, svc_pred)
lr_accuracy = accuracy_score(y_test, lr_pred)
nb_accuracy = accuracy_score(y_test, nb_pred)
xgb_accuracy = accuracy_score(y_test, xgb_pred)

print("Support Vector Classifier Accuracy:", svc_accuracy)
print("Logistic Regression Accuracy:", lr_accuracy)
print("Multinomial Naive Bayes Accuracy:", nb_accuracy)
print("XGB Accuracy:", xgb_accuracy)

In [ ]:
tmdb_tfidf = tfidf.transform(tmdb_above_100['word_soup'])
# print(soup_tfidf.shape,type(soup_tfidf))
tmdb_tfidf = tmdb_tfidf.astype('float32').toarray()
# 1 for F, 0 for M
tmdb_gender_pref = xgb_clf.predict(tmdb_tfidf)
tmdb_gender_pref.shape

In [ ]:
numerical_columns = ['vote_average', 'vote_count', 'revenue', 'runtime', 'budget',
                     'popularity', 'gender_pref','adult']
# converting back to -1 for male, 1 for female
tmdb_above_100['gender_pref'] = tmdb_gender_pref
tmdb_above_100['gender_pref'] = tmdb_above_100['gender_pref'].apply(lambda x : -1 if x == 0 else 1)
# -1 if adult, 1 if not
tmdb_above_100['adult'] = tmdb_above_100['adult'].apply(lambda x : -1 if x  else 1)
# without scaling first
tmdb_num = tmdb_above_100[numerical_columns]

In [ ]:
#free up ram
import gc
del df_ml_combined
del df_movies
del df_ratings
del df_users
del pre_2000
del pre_2000_analysis
del soup_tfidf
del tmdb_df
del merged_df
del xgb_clf
del svc_clf
del nb_clf
del lr_clf
del X_test
del X_train
del agg_analysis

gc.collect()

In [ ]:
# create cosine similarity matrix
# doing it in batches to free up memory


# def batch_cosine_similarity(m1, m2, batch_size=100):
#     assert m1.shape[1] == m2.shape[1]
#     ret = np.ndarray((m1.shape[0], m2.shape[0]), dtype=np.float32)
#     for row_i in range(0, int(m1.shape[0] / batch_size) + 1):
#         start = row_i * batch_size
#         end = min([(row_i + 1) * batch_size, m1.shape[0]])
#         if end <= start:
#             break
#         rows = m1[start: end]
#         sim = cosine_similarity(rows, m2) # rows is O(1) size
#         ret[start: end] = sim
#     return ret


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix_initial = cosine_similarity(tmdb_num.values,tmdb_num.values,dense_output = False)

In [ ]:
# threshold -when is it too unsimilar
# strict_k - when you want to enforce threshold
def recommend_movies(data, movie, similarity_matrix, top_k=10, threshold = 0.5, strict_k = True):
  # get index of movie with <title> in dataframe
  index = data.index[data['title'] == movie][0]
  try:
    score_arr = similarity_matrix[index].toarray()
  except:
    score_arr = similarity_matrix[index]
  if not strict_k:
    score_arr =   np.fromiter((x for x in score_arr if x > threshold), dtype=score_arr.dtype)
  reversed_score_arr = np.argsort(score_arr)[::-1]

  top_k_movies = reversed_score_arr[1:top_k + 1]
  return data.iloc[top_k_movies]

def recommend_movie_index(index, similarity_matrix, top_k=10, threshold = 0.5):
  try:
    score_arr = similarity_matrix[index].toarray()
  except:
    score_arr = similarity_matrix[index]

  score_arr =   np.fromiter((x for x in score_arr if x > threshold), dtype=score_arr.dtype)
  top_k = min(top_k,len(score_arr))
  reversed_score_arr = np.argsort(score_arr)[::-1]
  return reversed_score_arr[1:top_k + 1]



In [ ]:
recommend_movies(tmdb_above_100, "Inception", cos_matrix_initial)

In [ ]:
# gets all relevant movie indexes for 1 movie --> change rating to some other combination of variable

def get_relevant(df,ratings,index, a = 0.1):

    movie_score = df.iloc[index].sum()
    relevant = ratings.between(movie_score - a, movie_score + a).index
    return relevant


In [ ]:
# gets precision for 1 movie

def precision_for_movie(df,ratings,index,similarity_matrix,relevance_range):

    relevant_items = get_relevant(df,ratings,index,0.1)

    recommended_items = recommend_movie_index(index, similarity_matrix,
                                              top_k = len(relevant_items),
                                              threshold= 0.1)
    # Calculate the intersection of recommended_items and relevant_items
    true_positive = len(set(recommended_items).intersection(set(relevant_items)))

    # Calculate the total number of recommended items
    total_recommended_items = len(recommended_items)

    # Calculate precision
    precision_value = true_positive / total_recommended_items if total_recommended_items > 0 else 0
    return precision_value

# get model precision
def model_precision(df,ratings,similarity_matrix,relevance_range):
    total = 0
    df_dict = df.to_dict('records')
    for row in range(len(df_dict)):
        total += precision_for_movie(df,ratings,row, similarity_matrix,relevance_range)
    return total/len(df)

# get recall for 1 movie
def recall_for_movie(df,ratings,index,similarity_matrix,relevance_range):
    relevant_items = get_relevant(df,ratings,index,0.1)
    recommended_items = recommend_movie_index(index, similarity_matrix,
                                              top_k = len(relevant_items),
                                              threshold=0.1)
    # Calculate the intersection of recommended_items and relevant_items
    true_positive = len(set(recommended_items).intersection(set(relevant_items)))

    # Calculate the total number of relevant items
    total_relevant_items = len(relevant_items)

    # Calculate recall
    recall_value = true_positive / total_relevant_items if total_relevant_items > 0 else 0
    return recall_value
# get model recall
def model_recall(df,ratings,similarity_matrix,relevance_range):
    total = 0
    df_dict = df.to_dict('records')
    for row in range(len(df_dict)):
        total += recall_for_movie(df,ratings,row,similarity_matrix,relevance_range)
    return total/len(df)


def model_f1_score(precision,recall):
    return (2*precision*recall)/(precision+recall)


In [ ]:
tmdb_rating_initial = tmdb_num.sum(axis = 1)

In [ ]:
#compare precision, recall, and f1 before and after processing + additional feature engineering
precision_initial = model_precision(tmdb_num, tmdb_rating_initial,cos_matrix_initial,100000)
recall_initial = model_recall(tmdb_num,tmdb_rating_initial, cos_matrix_initial,100000)
f1_initial = model_f1_score(precision_initial,recall_initial)


In [ ]:
print(f'Model Precision: {precision_initial}')
print(f'Model Recall: {recall_initial}')
print(f'Model F1: {f1_initial}')

In [ ]:
# cos matrix with scaling and preprocessing here + add encoding
# scaling num columns
from sklearn.preprocessing import MinMaxScaler
to_scale_1 = ['vote_average', 'revenue', 'runtime', 'budget',
             'gender_pref','adult']
            #  female pref non adult movie -> 9 --> because non adult is +ve and female pref is +ve
            #  male rpef adult movie --> 6
            # sum(numerical columns) * gender pref --> female pref movies --> rating +ve and -ve for men
# ['vote_average' + 'revenue' + 'runtime'+ 'budget'+ 'gender_pref' +'adult' + 'vote_count'+'popularity']
to_scale_2 = ['vote_count','popularity']
tmdb_above_100_copy = tmdb_above_100.copy(deep=True)
scaler1 = MinMaxScaler(feature_range = (-1,1))
# vote_count and popularity more correlated with higher vote average
scaler2 = MinMaxScaler(feature_range = (-2,2))
# tmdb_above_100_copy.head()

tmdb_above_100['gender_pref'] = tmdb_above_100['gender_pref'].apply(lambda x : -1 if x == 0 else 1)
tmdb_above_100_copy[to_scale_1] = scaler1.fit_transform(
    tmdb_above_100_copy[to_scale_1])
tmdb_above_100_copy[to_scale_2] = scaler2.fit_transform(
    tmdb_above_100_copy[to_scale_2])
# add encoding back

tmdb_scaled = pd.concat([tmdb_above_100_copy[numerical_columns], pd.DataFrame(tmdb_tfidf)], axis=1)
tmdb_scaled.head()


In [ ]:
tmdb_rating_scaled = tmdb_scaled[numerical_columns].sum(axis = 1)

In [ ]:
del cos_matrix_initial
gc.collect()

In [ ]:
cos_matrix_final = cosine_similarity(tmdb_scaled.values,tmdb_scaled.values,dense_output = False)

precision_final = model_precision(tmdb_scaled,tmdb_rating_scaled,cos_matrix_final)
recall_final = model_recall(tmdb_scaled,tmdb_rating_scaled,cos_matrix_final)
f1_final = model_f1_score(precision_final, recall_final)

In [ ]:
print(f'Model Precision: {precision_final}')
print(f'Model Recall: {recall_final}')
print(f'Model F1: {f1_final}')

## Final Recommendation

In [ ]:
import random
def add_random_movie(random_movie_df, recommended_movies_df,n_random = 1):
  random.seed(42)
  while n_random:
    random_index = random.randint(0, random_movie_df.shape[0])
    recommended_movies_df = recommended_movies_df.append(
        random_movie_df.iloc[random_index])
    n_random -= 1
  return recommended_movies_df

# include random movie from tmdb_below_100 to improve serendipity
recommended_movies = add_random_movie(tmdb_below_100,
                                      recommend_movies(tmdb_above_100,
                                      "Inception", cos_matrix_final))


# End of work